# Collecting Tweets using Tweepy

In [1]:
# Load packages
import tweepy as tw
import pandas as pd
import numpy as np
from preprocessor import api as p #to clean tweets

In [2]:
# Credentials
access_token = "1119973522435035136-6FjyeGe4XlblWuaNNTIzwhjoNBxsrA"
access_token_secret = "NNYIIsZtlZc6YJpgbEbPWl9ecPfotympspjIIjrXVjWEp"
api_key = "A3koB4EuhkEfST1fcYizS2CHY"
api_key_secret = "WHo7GMPKTCJWNDmgaMBfsfZNFuLFR4Kp16aBwQJS7tKaSsnBHQ"

In [3]:
# Authentication app
auth = tw.OAuthHandler(api_key, api_key_secret)
auth.set_access_token(access_token, access_token_secret)
api = tw.API(auth)

In [21]:
# Find by phrase or keywords within a specified time
search_words = '"utah lake" -filter:links -filter:retweets' #filter out retweets and tweets with links

# advanced search in Twitter search box 
# utah lake (#utahlake) until:2020-12-03 since:2020-11-28 -filter:links -filter:replies
# If I include hashtag in search, it wouldn't show the ones without hashtag
# May be we can have a separate search by #utahlake and then combine both the dataframes

# From date to date (YYYY-MM-DD)
date_since = '2020-11-28'
date_until = '2020-12-3'

# Create a cursor object that iterates over the requested information
tweets = tw.Cursor(api.search, q=search_words, lang="en",
                  since=date_since, until=date_until, 
                  tweet_mode='extended').items()

#when we have premium subscription we will use this
#API.search_full_archive(environment_name, query[, tag][, fromDate][, toDate][, maxResults][, next])

# Parse the cursor object into rows of lists using list comprehension
rows = [[tweet.user.location, tweet.full_text, tweet.created_at, tweet.coordinates] for tweet in tweets]

In [22]:
# Create dataframe from the rows of lists
df_tweet = pd.DataFrame(data=rows, columns=['location', 'tweet', 'time', 'coordinates'])

In [23]:
# Add ID column (because we removed user information)
unique_id = np.arange(len(df_tweet))
df_tweet['id'] = unique_id + 1
df_tweet = df_tweet[['id', 'location', 'tweet', 'time', 'coordinates']]

In [24]:
df_tweet.head()

,id,location,tweet,time,coordinates


In [19]:
df_tweet.tail()

,id,location,tweet,time,coordinates
0,1,Western USA,@senatorshoshana It is beautiful. I used to dr...,2020-12-01 03:44:56,None
1,2,"Seattle, WA",I asked about how UW compensates for the loss ...,2020-11-30 19:50:02,None
2,3,BYU,@bayoulejeune Provo river trail is actually so...,2020-11-30 01:49:52,None
3,4,"Provo, UT",@bayoulejeune Hopefully the Utah Lake trail in...,2020-11-30 01:36:28,None
4,5,,@RonPlacone Patience. While the current trend...,2020-11-29 03:08:27,None


In [13]:
tw.coordinates

AttributeError: module 'tweepy' has no attribute 'coordinates'

In [9]:
type(tweets)

tweepy.cursor.ItemIterator

# Cleaning tweets in the dataframe 

In [10]:
# Clean each tweet in our dataframe using the preprocessor module
tweet_clean = []
for tweet in df_tweet['tweet']:
    cleaned = p.clean(tweet)
    tweet_clean.append(cleaned)

In [11]:
# Store as a new dataframe
df_tweet_clean = df_tweet.copy()
df_tweet_clean['tweet'] = tweet_clean

In [12]:
df_tweet_clean.head()

,id,location,tweet,time
0,1,,Damn I was really disrespecting Utah once you ...,2020-11-25 23:37:16
1,2,,"'35.09""N '58.26""W based on the October data, i...",2020-11-25 23:04:02
2,3,"Salt Lake City, UT",Utah County from what I have had is kind of th...,2020-11-25 22:41:20
3,4,"Salt Lake City, UT",Called earlier today: I just wanted to call to...,2020-11-25 22:30:53
4,5,,I can see the dashboard now in an investor pre...,2020-11-25 22:28:59


# Save the dataframe as a CSV

In [24]:
# Checking current working directory
import os
os.getcwd()

'C:\\Users\\Peaceful Pierre\\Documents\\Academics\\Masters Project\\code'

In [26]:
# Store as CSV to a relative path
df_tweet_clean.to_csv("scraped_tweets_test.csv")